In [1]:
(ns linear-regression-multivariable
  (:require
            [clojure.data.fressian :as fress]
            [clojure.java.io :as io]
            [clojure.set :as set]
            [clojure.string :as str]))

In [2]:
(def j 0.000001) ; Defining the learning rate. 

#'linear-regression-multivariable/j

In [3]:
;# Function for symbolic differentiation #

(defn variable? [e] (symbol? e))

;Should be a symbol first. And should be equal. 

(defn same-variable? [v1 v2] 
 (and (variable? v1) (variable? v2) (= v1 v2)))

;Equality check of two numbers

(defn =number? [x n] 
 (and (number? x) (= x n)))

(defn make-sum [a1 a2]
 (cond (=number? a1 0) a2
       (=number? a2 0) a1
       (and (number? a1) (number? a2)) (+ a1 a2)
       :else (list '+ a1 a2)))

(defn make-product [m1 m2]
 (cond (or (=number? m1 0) (=number? m2 0)) 0
       (=number? m1 1) m2
       (=number? m2 1) m1
       (and (number? m1) (number? m2)) (* m1 m2)
       :else (list '* m1 m2)))

(defn sum? [x]
 (and (seq? x) (= (first x) '+)))

;To get the first part of sum expression.

(defn addend [s] 
 (second s))

(defn augend [s]
 (last s))

(defn minus? [x]
 (and (seq? x) (= (first x) '-)))

(defn substract [a1 a2]
 (cond (=number? a1 0) ('* '- a2)
       (=number? a2 0) a1
       (and (number? a1) (number? a2)) (- a1 a2)
       :else (list '- a1 a2)))

(defn product? [x]
 (and (seq? x) (= (first x) '*)))

(defn multiplier [p]
 (second p))

(defn multiplicand [p]
  (last p))

;Now to make exponential. 

(defn make-exponentiation [b e]
 (cond (=number? e 0) 1
       (=number? e 1) b
       (and (number? b) (number? e)) (Math/pow b e)
       :else (list '** b e)))

(defn exponentiation? [exp]
  (and (seq? exp) (= '** (first exp))))

(defn base [exp]
  (second exp))

(defn exponent [exp]
  (last exp))

(def trigonometry #{'sin 'cos 'tan})

(defn trigonometry? [exp]
    (contains? trigonometry (first exp)))

(defn deriv [exp var]  

 (cond (number? exp) 0 
       (variable? exp) (if (same-variable? exp var) 1 0)
     
       (sum? exp)
       (if (<= (count exp) 3)
       (make-sum (deriv (addend exp) var)
          (deriv (augend exp) var))
           
        ;else part 
        (make-sum (deriv (addend exp) var)
        (deriv (cons '+ (rest (rest exp))) var))
           )
       
       (minus? exp)
       (substract (deriv (addend exp) var)
          (list '- (deriv (augend exp) var)))
       
      (product? exp)
       (if (<= (count exp) 3)
       (make-sum
        (make-product (multiplier exp)
             (deriv (multiplicand exp) var))
            (make-product (deriv (multiplier exp) var)
               (multiplicand exp)))
           
         ;else part   
        (make-sum
        (make-product (multiplier exp)
             (deriv (cons '* (rest (rest exp))) var))
             (make-product (deriv (multiplier exp) var)
              (cons '* (rest (rest exp)))
               ;(rest (rest exp))
                          ))
           )
       
       
       (exponentiation? exp)
         (make-product 
          (make-product (deriv (base exp) var) (exponent exp))
          (make-exponentiation (base exp) (dec (exponent exp)))) 
       
       (trigonometry? exp)
          (let [rest-exp (if (= (count (rest exp)) 1) (first (rest exp)) (rest exp))]
                 (cond (= (first exp) 'sin) (list '* (list 'cos rest-exp) (deriv rest-exp var))
                       (= (first exp) 'cos) (list '* (- 1) (list 'sin rest-exp) (deriv rest-exp var))))
      :else
     (prn "Invalid Operation")))


#'linear-regression-multivariable/deriv

In [28]:
;# Creating a function that takes two vectors and calculates the dot products of two vectors.
;# It calculates the dot product depending on if the values are numeric or symbol.
;# For numeric vectors, it returns a numeric scalar value and for symbol it returns a list like (+ (* x1 m1) (* x2 m2))


(defn dot-symbol [v1 v2]
    (if (or (symbol? (first v1)) (symbol? (first v2)))
        (conj (map #(list '* %1 %2)  v1 v2) '+)
        (reduce + (map * v1 v2))
        ))

(defn ** [x y]
    (Math/pow x y))

(defn min-max [v]
    (let [min-num (apply min v)
          max-num (apply max v)
          ]
        (map #(float (/ (- % min-num) (- max-num min-num))) v)))

;Creating a function that calculates the tranpose of a matrix. 

(defn transpose [m]
    (apply map vector m)
    )

;Creating a function that normalizes the data using minimum and maximum value.

(defn normalize [data]
    (transpose (map #(min-max %) (transpose data)))
    )

#'linear-regression-multivariable/normalize

In [5]:
(prn (dot-symbol [1 2 3] [4 5 6]))
(prn (dot-symbol '[x1 x2] '[m1 m2]))

32
(+ (* x1 m1) (* x2 m2))


In [6]:
;# Creating two functions that create features and coefficients, depending on the size of n (which is size of data).

;# So it returns values like (x1 x2 x3 x4 x5) and (m1 m2 m3 m4 m5) for n = 5

(defn create-features [n]
    (map read-string (map #(str "x" %) (range 1 (+ n 1))))
    )

(defn create-coefficients [n]
    (map read-string (map #(str "m" %) (range 1 (+ n 1))))
    )

#'linear-regression-multivariable/create-coefficients

In [7]:
(prn (create-features 5))
(prn (create-coefficients 5))

(x1 x2 x3 x4 x5)
(m1 m2 m3 m4 m5)


In [8]:
;# Creating two functions that assigns values to the features and the coefficients. 

(defn assign-features [n features]
     (map #(str "(def x" %1 " " %2 ")") (range 1 (+ n 1)) features)
    )

(defn assign-coefficients [n coefficients]
     (map #(str "(def m" %1 " " %2 ")") (range 1 (+ n 1)) coefficients)
    )

#'linear-regression-multivariable/assign-coefficients

In [29]:
(defn linear-regression-multiple [data]
    
    ;Initalizing m (which is a vector of coeffcients) with 0.5
    (def m (vec (repeat (count (rest (first data)) ) 0.5))) 
    (def error 0)
    
    ;Running a loop across the whole data
    (loop [d  data]
        (when (> (count  d) 1)
            
            
            (do (def y (first (first d)));First element of the list is considered as y. 
                (def x (rest (first d)));Rest of the elements of the list is considered as x. (x is a vector)
                
                ;Creates x-features (x1 x2 ..xn) depending on the count of data.
                (def x-features (create-features (count m)))
              
                ;Creates m-coefficients (m1 m2 ...mn) depending on the count of data.
                (def m-coefficients (create-coefficients (count m)))
                
                ;Calculates m1x1 + m2x2 + .... mxxn, which I have called loss-distance. 
                (def loss-distance (dot-symbol m-coefficients x-features))
                
                ;Calculates the loss function. 
                (def loss-function (read-string (str "(** (- y " 
                                                     (dot-symbol (create-coefficients (count m))
                                                                    (create-features (count m))) ") 2)")))
                
                
                ;Calculates the derivate of loss function by the vector of coefficients.
                ;I have used map operation and anonymous function to calculate derivate of loss function by a vector.
                (def deriv-expressions (map #(deriv loss-function %) m-coefficients))
                    
                ;Assigns the calculated value of m and x (both vectors) to the symbols obtained above.
                ;Again using the map operation to loop over a vector.
                
                ;eval operation evaluates the value of the symbolic expression. 
                
                ;Also, we need to perform the side effect inside this map operation as map produces lazy sequence only.
                
                ;Therefore, we use dorun operation here, which performs side effect in clojure map operation. 
                (dorun (map eval (map read-string (assign-coefficients (count m) m))))
                (dorun (map eval (map read-string (assign-features (count m) x))))
                
                ;Using eval operation again to evaluate the value
                (def ans (map eval deriv-expressions))
                
                ;Also calculating and updating our error value.
                (def error-evaluated (eval loss-function))
                (def error (+ error error-evaluated))
                
                ;Final calculation of m vector, which is our coefficient. 
                ;This calculated value of m vector is updated in every iteration. 
                (def m (map + (map #(* (- j) %) ans) m))
            )
            (recur (rest d))))
    (do 
        (prn "The final calculated coefficient vector is: ", m)
    
    (prn "The final error is: ", error)
    )
         ; Returning the final value of m that is our vector of coefficients. 
    )

#'linear-regression-multivariable/linear-regression-multiple

In [23]:
;Taking a dummy data to check our function. 
(def data 
'([68.78190405 162.3104725 100 ] [74.11010539 212.7408556 120 ] [71.7309784 220.0424703 120 ] [69.88179586 206.3498006 120 ] [67.25301569 152.2121558 120 ] 
  [68.78508125 183.9278886 120 ] [68.34851551 167.9711105 120 ]))



#'linear-regression-multivariable/data

In [24]:
(linear-regression-multiple data)

"The final calculated coefficient vector is: " (0.34735170659248005 0.4087745367081262)
"The final error is: " 26878.44318839454


In [12]:
;Now, using the data that we used previous week to check the new multiple linear function.
; We will then compare our result from this new multiple linear function with that of our result from last week 
; using simple linear function using the same data. 
(def last-week-data 
'([68.78190405 162.3104725] [74.11010539 212.7408556] [71.7309784 220.0424703] [69.88179586 206.3498006] [67.25301569 152.2121558] [68.78508125 183.9278886] [68.34851551 167.9711105] [67.01894966 175.9294404] [63.45649398 156.3996764] [71.19538228 186.6049256] [71.64080512 213.7411695] [64.76632913 167.1274611] [69.2830701 189.4461814] [69.24373223 186.434168] [67.6456197 172.1869301] [72.41831663 196.0285063] [63.97432572 172.8834702] [69.6400599 185.9839576] [67.93600485 182.426648] [67.91505019 174.1159291] [69.43943987 197.7314216] [66.14913196 149.173566] [75.20597361 228.7617806] [67.89319634 162.0066518] [68.1440328 192.3439766] [69.08963143 184.4351744] [72.80084352 206.8281894] [67.42124228 175.2139224] [68.49641536 154.3426389] [68.61811055 187.5068432] [74.03380762 212.9102253] [71.52821604 195.0322432] [69.1801611 205.1836213] [69.57720237 204.1641255] [70.40092889 192.9035151] [69.07617117 197.4882426] [67.19352328 183.8109732] [65.80731565 163.8518249] [64.30418789 163.1080171] [67.97433623 172.1355974] [72.18942596 194.0454049] [65.27034552 168.6177462] [66.09017738 161.1934326] [67.51032152 164.6602773] [70.10478626 188.9223032] [68.25183644 187.0605522] [72.17270912 209.0708634] [69.17985762 192.0143354] [72.87036015 211.3424968] [64.78258298 165.6116262] [70.18354989 201.0719181] [68.49145025 173.4239603] [67.33083088 181.4076793] [66.99094408 169.7377074] [66.4995499 163.3095283] [68.35305665 189.7102099] [70.77445907 192.1248473] [71.21592367 198.1984641] [70.01336535 209.5264998] [71.40318222 198.7597927] [69.55200509 198.0795237] [73.81853456 195.2906122] [66.99688275 164.943303] [71.41846589 179.8639023] [65.27930021 155.2504207] [68.27419147 184.5193912] [72.76536995 220.6780415] [68.0993798 183.3126553] [68.89670607 196.4513123] [69.28950996 184.5956083] [70.52322452 207.5328385] [69.66372523 177.2009288] [67.59526881 163.1080022] [72.50812038 216.2182301] [71.2529856 204.6554938] [71.80918689 200.9205712] [72.24516548 220.9017699] [66.51262766 196.4498603] [66.029034 168.64081] [67.57715394 181.432713] [68.2465686 198.6586733] [73.826127 237.9167359] [69.80246436 173.0412559] [65.95957778 160.6839166] [71.07901758 188.6029191] [66.59619654 208.345694] [68.95153509 193.4351185] [68.24446179 174.1096938] [72.31682512 197.3685892] [71.81542045 201.6206667] [65.23704952 181.0119732] [70.64053009 182.122486] [64.7319256 177.5492634] [67.10355118 164.9745807] [65.11748489 165.7171122] [71.70123402 193.0941635] [66.83287821 180.683887] [66.47127526 172.7737229] [69.41152622 177.4706162] [70.05217747 200.8187373] [66.74360465 167.7397753] [66.27432912 154.3956248] [68.32844799 177.9847286] [70.0758882 183.938857] [68.73298815 179.2047821] [67.55605126 158.5679874] [66.25363253 166.5035316] [69.18220268 171.9836776] [67.60910494 174.3641824] [69.29273802 187.7453519] [68.19068401 201.4879624] [71.6070858 204.9270121] [69.19685751 178.6855465] [67.26196098 164.0144277] [73.6851934 223.2700044] [69.53721501 202.9583063] [68.31155984 189.3917856] [67.73896347 178.1705556] [71.7057626 223.2993352] [63.6322646 167.0287067] [68.72119846 180.468682] [66.94934165 164.4391456] [62.70698974 149.9696184] [72.25840892 192.9102666] [70.90865306 201.8839987] [67.6098436 179.868026] [70.80155896 196.4677247] [69.30476905 187.5516495] [66.24289834 171.6931597] [67.49219298 191.6996048] [65.80624829 165.8506478] [71.44370566 185.6627413] [68.46440582 178.2818007] [63.9879246 150.9790513] [71.00189769 209.9293234] [68.13972419 174.1578043] [68.39540025 179.8789771] [68.09621975 167.7471412] [68.14059036 183.0446604] [68.86009031 169.5568208] [66.14885254 173.4316727] [66.20603205 180.8895546] [67.43212021 183.5089329] [69.47110603 205.9779102] [70.51585969 183.7598452] [71.33837604 201.9544955] [71.00194477 216.1200947] [66.20234771 159.3903237] [72.54330705 210.3364247] [67.47935176 178.6183502] [65.35041056 146.7397976] [70.84406242 195.937015] [69.93847526 170.1758538] [64.73981548 169.6540541] [69.30840288 198.3786936] [68.83846286 179.9077169] [61.93732327 147.2636865] [68.59333554 176.9842511] [65.21857558 185.4677884] [64.33364811 177.4938023] [68.7489067 169.8274774] [72.4896554 198.5816982] [67.23393092 203.3503393] [67.26360484 193.2176727] [65.11850428 178.1349452] [66.26282004 162.7795834] [67.70167966 159.2865106] [65.53069597 172.114312] [69.86896981 201.8370426] [68.48187536 170.3523296] [72.21396335 205.3897607] [68.17953269 169.7877681] [71.98120654 200.4373105] [66.06513673 178.588954] [66.65616417 168.0140297] [67.5994242 177.7664258] [68.24594409 193.277624] [64.80862144 162.3795406] [67.49221827 183.3838244] [68.18073071 181.0560953] [69.5533849 210.871799] [66.40224967 165.5684991] [66.59215711 196.1484498] [71.93588658 202.0464252] [68.28704173 170.4136025] [69.9554512 201.9480853] [71.85112915 193.3869846] [65.75549864 179.8556896] [67.03185208 156.489935] [76.70983486 235.0354188] [72.57112137 230.560535] [69.7288049 203.895655] [72.799224 218.2163966] [72.53935407 222.6671779] [72.29474338 208.4748659] [67.25332482 180.77978] [75.94446038 231.9247485] [66.31623192 170.5938581] [71.15605349 188.2714563]))
 

#'linear-regression-multivariable/last-week-data

In [25]:
(linear-regression-multiple last-week-data)

"The final calculated coefficient vector is: " (0.34202441721006666)
"The final error is: " 8345.509965436779


In [14]:
;0.35349640499237406, the value we achieved last week using linear-regression-with-symbolic. 
;Now, we get 0.342
;So, almost same!
;That is, now performing Simple Linear Regression with also this Multiple Linear Regression algorithm.

In [15]:
;;; Now, we will use one well known data to calculate the coefficients using our function.
;;; Since our function can take any size of input features, we will use a dataset with more than 3 features.

;;; We will use motor cars dataset with miles per gallon (mpg) as output variable, and displacement (disp), horsepower
;;; (hp), drat and weight (wt) as input variables or features. 


;;; Functions to read csv file in clojure. 

;; (defn line->stype

;;   [line]
;;   (let [[a b c d e] (str/split line #"[,]")]
;;     [(read-string a) (read-string b) (read-string c) (read-string d) (read-string e)]))

;; (defn load-data
;;   []
;;   (if (.exists (io/file "resources/mtcars.csv"))
;;     (->> (io/resource "mtcars.csv")
;;          (io/reader)
;;          (line-seq)
;;          (map #(line->stype %)))
;;     nil))



In [26]:
;;; Use this in case you have problem loading csv data using fressian. 

(def mtcars-data '([mpg disp hp drat wt] [21 160 110 3.9 2.62] [21 160 110 3.9 2.875] [22.8 108 93 3.85 2.32] [21.4 258 110 3.08 3.215] [18.7 360 175 3.15 3.44] [18.1 225 105 2.76 3.46] [14.3 360 245 3.21 3.57] [24.4 146.7 62 3.69 3.19] [22.8 140.8 95 3.92 3.15] [19.2 167.6 123 3.92 3.44] [17.8 167.6 123 3.92 3.44] [16.4 275.8 180 3.07 4.07] [17.3 275.8 180 3.07 3.73] [15.2 275.8 180 3.07 3.78] [10.4 472 205 2.93 5.25] [10.4 460 215 3 5.424] [14.7 440 230 3.23 5.345] [32.4 78.7 66 4.08 2.2] [30.4 75.7 52 4.93 1.615] [33.9 71.1 65 4.22 1.835] [21.5 120.1 97 3.7 2.465] [15.5 318 150 2.76 3.52] [15.2 304 150 3.15 3.435] [13.3 350 245 3.73 3.84] [19.2 400 175 3.08 3.845] [27.3 79 66 4.08 1.935] [26 120.3 91 4.43 2.14] [30.4 95.1 113 3.77 1.513] [15.8 351 264 4.22 3.17] [19.7 145 175 3.62 2.77] [15 301 335 3.54 3.57] [21.4 121 109 4.11 2.78]))



#'linear-regression-multivariable/mtcars-data

In [17]:
;(def mtcars-data (drop 1 (load-data)))

In [30]:
;;; This part we performed to compare our result with non linear regression. (Basically comparing the error.)

(linear-regression-multiple (normalize (drop 1 mtcars-data)))

"The final calculated coefficient vector is: " (0.4999839215052779 0.4999867851580205 0.49999523565657705 0.49998443109115)
"The final error is: " 13.053241688148475
